### Simulation a 2 dimensional Ising model on square lattice with or without frustrations
The simulation is a single spin flip Monte Carlo Metropolis algorithm.
To start the simulation call IsingSimulate() function

MCM steps - Monte Carlo Metropolis steps
```
IsingSimulate() function
parameters:
    L - linear lattice size
    T - temperature value
    sweeps - number of MCM steps
    T_corr - number of MCM steps between configuration records in the output array
    J -  value of spins neigbours interaction
    Jd - value of diagonal spins neigbours interaction
    seed - seed for mc_lib rndm initialisation
    rseed - seedSequence for mc_lib rndm initialisation

outputs:
    configs
    e - mean energy for the whole simulation
    m - mean magnetisation
```
Example [https://colab.research.google.com/drive/1Xwsx39wkBaopYTNswiy2OenMIx5lYBLZ?usp=sharing](https://colab.research.google.com/drive/1Xwsx39wkBaopYTNswiy2OenMIx5lYBLZ?usp=sharing)


In [ ]:
# Check if mc_lib installed
try:
    from mc_lib.rndm import RndmWrapper
except ImportError:
    !pip install git+https://github.com/ev-br/mc_lib.git@master
    pass

In [ ]:
%load_ext cython

In [ ]:
%%cython --cplus
import cython
import numpy as np
cimport numpy as np
from libc.math cimport exp
from mc_lib.rndm cimport RndmWrapper
from mc_lib.lattices import tabulate_neighbors
from mc_lib.observable cimport RealObservable

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
cdef np.ndarray initState(long[::1] lattice,
                                    RndmWrapper rndm):
"""

:param   lattice: unconfigured array of future lattice (empty array)
:param   rndm: class with random methods from mc_lib
:return: change the 'Lattice' variable and return nothing
"""
for i in range(lattice.shape[0]):
    lattice[i] = 1 if rndm.uniform() > 0.5 else -1
return


@cython.boundscheck(False)
@cython.wraparound(False)
cdef np.ndarray mcmove(long[::1] config,
                                 double beta,
                                        long[:, ::1] ngb,
                                                     RndmWrapper rndm,
                                                                 const double[:,::1] Js):
"""
One flip attempt
:param    config: Current configuration of lattice
:param    beta:   Inversed temperature of current configuration
:param    L:      Linear size 'L' of lattice
:param    ngb:    Array of neigbours
"""
cdef:
Py_ssize_t site = int(config.shape[0] * rndm.uniform())
Py_ssize_t site1 = 0
double dE = 0
long num_ngb = ngb[site, 0]
for n in range(1, num_ngb + 1):
    site1 = ngb[site, n]
    dE += config[site1] * config[site] * Js[site,site1]
cdef double ratio = exp(-2 * dE * beta)
if rndm.uniform() > ratio:
    return
config[site] *= -1
return


@cython.boundscheck(False)
@cython.wraparound(False)
cdef double calcEnergy(long[::1] config,
                                 long[:, ::1] ngb,
                                              const double[:,::1] Js):
"""
    Count current configuration energy
    :param    config: Current configuration of lattice
    :param    ngb:    Array of neigbours
    return:
"""
cdef:
Py_ssize_t site = 0
Py_ssize_t site1 = 0
double energy = 0
for site in range(config.shape[0]):
    num_ngb = ngb[site, 0]
    for n in range(1, num_ngb+1):
        site1 = ngb[site, n]
        energy += -1 * config[site] * config[site1] * Js[site, site1]
return energy / 4.


def get_J( double[:,::1] Js, double J, double Jd, int L1, int L2):
"""Tabulate the couplings per site."""
cdef Py_ssize_t i
for i in range(L1*L2):
    Js[i, ((i // L2 + 1) % L1 * L2 )  + (i + 1) % L2 ] = Jd
    Js[i, ((i // L2  - 1) % L1 * L2 )  + (i - 1) % L2 ] = Jd
    Js[i, (i // L2) * L2 + (i + 1) % L2] = J
    Js[i, (i + L2) % (L1*L2)] = J
    Js[i, (i // L2) * L2 + (i - 1) % L2] = J
    Js[i, (i - L2) % (L1*L2)] = J
return


@cython.boundscheck(False)
@cython.wraparound(False)
def IsingSimulate(L, T, sweeps, int T_corr, double J=1, double Jd=0, int seed = np.random.randint(0, 1000), int rseed = 1234):
"""
    L - linear_size
    T - One temperature point
    J - value of spins neigbours interaction
    Jd - value of diagonal spins neigbours interaction
    Sweeps - number of L^2 Metropolis Monte-Karlo steps
"""
cdef RndmWrapper rndm = RndmWrapper((rseed, seed))
cdef:
float beta = 1.0 / T
int sweep = 0
int steps_per_sweep = L * L
int num_therm = int(12 * L * L)
int i = 0

cdef:
np.ndarray configs = np.empty([sweeps, L*L], dtype=int)
long[::1] config = np.empty(L*L, dtype=int)
int steps = 0
long[:, ::1] ngb
if Jd == 0:
    ngb = tabulate_neighbors((L, L, 1), kind='sc')
else:
    ngb = tabulate_neighbors(L, kind='triang')

cdef double[:,::1] Js = np.zeros((L*L, L*L))
get_J(Js, J, Jd, L, L)

cdef RealObservable m1 = RealObservable()
cdef RealObservable e1 = RealObservable()

initState(config, rndm)
for sweep in range(num_therm):
    mcmove(config, beta, ngb, rndm, Js)


for sweep in range(sweeps):
    for i in range(T_corr):
        steps += 1
        for i in range(steps_per_sweep):
            mcmove(config, beta, ngb, rndm, Js)
    configs[sweep] = config.copy()
    energ = calcEnergy(config, ngb, Js)
    magn = np.sum(config)
    e1.add_measurement(energ)
    m1.add_measurement(magn)

return configs, e1.mean, m1.mean

In [ ]:
IsingSimulate(L=4, T=2.26, sweeps=100, T_corr=1, J=1, Jd=0, seed=10, rseed=10)

## Energy and magnetisation graphs

In [ ]:
%%time
import numpy as np
ms = []
es = []
configs = []
L = 20
Temps = np.linspace(1.8, 3, 30)
for i, t in enumerate(Temps):
    config, e, m = IsingSimulate(L=L, T=t, sweeps=10000, T_corr=50, J=1, Jd=0, seed=0)
    es.append(e)
    ms.append(m)
    configs.append(config)
    if i%10 == 0:
        print(t)
es = np.array(es)
ms = np.array(ms)
configs = np.array(configs)

In [ ]:
try:
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(7, 5))
    fontsize = 16
    T_c = 2 / (np.log(1 + 2 ** 0.5))
    plt.scatter(Temps, es / L**2, marker = "o", linestyle="None", label='E(T)')
    plt.axvline(T_c, color="tab:orange")
    plt.legend(fontsize=fontsize)
    plt.xlabel("T", fontsize=fontsize)
    plt.ylabel(r"E(t) / $L^2$", fontsize=fontsize)
    plt.grid()
    fig = plt.figure(figsize=(7, 5))
    plt.scatter(Temps, np.abs(ms / L**2), marker = "o", linestyle="None", label='M(T)')
    plt.axvline(T_c, color="tab:orange")
    plt.legend(fontsize=fontsize)
    plt.xlabel("T", fontsize=fontsize)
    plt.ylabel(r"M(t) / $L^2$", fontsize=fontsize)
    plt.grid()
except ImportError:
    print('You need matplotlib be installed')
